In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def search_restaurant(restaurant_name):
    # 웹드라이버 설정
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    # 네이버 지도 페이지 열기
    driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
    
    try:
        # 검색 입력 필드가 로드될 때까지 대기
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
        )
        
        # 검색어 입력
        search_box.send_keys(restaurant_name)
        search_box.send_keys(Keys.RETURN)
        
        # 검색 결과가 로드될 때까지 대기 (여기서는 간단히 3초 대기)
        time.sleep(3)
        
        # 여기에 검색 결과를 처리하는 코드를 추가할 수 있습니다.
        # 예: 검색 결과 목록 가져오기, 특정 정보 추출 등
        
    except Exception as e:
        print(f"오류 발생: {e}")
    
    finally:
        # 브라우저 종료
        driver.quit()

# 사용 예
search_restaurant("맛불")

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def setup_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저를 표시하지 않고 실행
    return webdriver.Chrome(service=service, options=options)

def extract_reviews(driver, url, num_reviews=10):
    driver.get(url)
    reviews = []
    
    try:
        # 리뷰 섹션으로 이동
        review_tab = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='review']"))
        )
        review_tab.click()
        
        # 리뷰 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.pui__X35jYm"))
        )
        
        while len(reviews) < num_reviews:
            review_elements = driver.find_elements(By.CSS_SELECTOR, "li.pui__X35jYm")
            
            for element in review_elements:
                if len(reviews) >= num_reviews:
                    break
                
                username = element.find_element(By.CSS_SELECTOR, "span.pui__NMi-Dp").text
                review_text = element.find_element(By.CSS_SELECTOR, "a.pui__xtsQN-").text
                
                review_stats = element.find_elements(By.CSS_SELECTOR, "span.pui__WN-kAf")
                review_count = review_stats[0].text if len(review_stats) > 0 else ''
                photo_count = review_stats[1].text if len(review_stats) > 1 else ''
                
                keywords = [span.text for span in element.find_elements(By.CSS_SELECTOR, "span.pui__V8F9nN")]
                keywords_str = ', '.join(keywords)
                
                visit_info = element.find_element(By.CSS_SELECTOR, "div.pui__QKE5Pr")
                visit_date = visit_info.find_element(By.TAG_NAME, "time").text
                visit_count = visit_info.find_elements(By.CSS_SELECTOR, "span.pui__gfuUIT")[1].text
                
                reviews.append({
                    'Username': username,
                    'Review Text': review_text,
                    'Review Count': review_count,
                    'Photo Count': photo_count,
                    'Keywords': keywords_str,
                    'Visit Date': visit_date,
                    'Visit Count': visit_count
                })
            
            # 다음 페이지로 이동
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.pui__HK56z")
                next_button.click()
                time.sleep(2)  # 페이지 로딩 대기
            except:
                break  # 다음 페이지가 없으면 종료
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return reviews

def save_to_csv(data, filename='naver_map_reviews.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

# 메인 실행 코드

url = "https://map.naver.com/p/smart-around/place/1647171960?c=15.00,0,0,0,dh&placePath=/review"  # 실제 장소 URL로 변경해야 함
driver = setup_driver()
reviews = extract_reviews(driver, url, num_reviews=50)  # 원하는 리뷰 수 지정
driver.quit()
save_to_csv(reviews)

An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00656AB3+25587]
	(No symbol) [0x005E9C54]
	(No symbol) [0x004E2113]
	(No symbol) [0x00526F62]
	(No symbol) [0x005271AB]
	(No symbol) [0x00567852]
	(No symbol) [0x0054ABE4]
	(No symbol) [0x00565370]
	(No symbol) [0x0054A936]
	(No symbol) [0x0051BA73]
	(No symbol) [0x0051C4CD]
	GetHandleVerifier [0x00934C63+3032483]
	GetHandleVerifier [0x00986B99+3368153]
	GetHandleVerifier [0x006E8F62+624802]
	GetHandleVerifier [0x006F07DC+655644]
	(No symbol) [0x005F260D]
	(No symbol) [0x005EF6D8]
	(No symbol) [0x005EF875]
	(No symbol) [0x005E1CA6]
	BaseThreadInitThunk [0x7701FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77C480CE+286]
	RtlGetAppContainerNamedObjectPath [0x77C4809E+238]

Data saved to naver_map_reviews.csv


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

def setup_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    return webdriver.Chrome(service=service, options=options)

def wait_and_find_element(driver, by, value, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        return element
    except TimeoutException:
        print(f"Element not found: {value}")
        return None

def search_and_extract_reviews(restaurant_name, num_reviews=10):
    driver = setup_driver()
    reviews = []
    
    try:
        # 네이버 지도 페이지 열기
        driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
        
        # 검색 입력 필드가 로드될 때까지 대기
        search_box = wait_and_find_element(driver, By.CSS_SELECTOR, "input.input_search")
        if not search_box:
            print("Search box not found")
            return reviews
        
        # 검색어 입력
        search_box.send_keys(restaurant_name)
        search_box.send_keys(Keys.RETURN)

        time.sleep(3)  # 검색 결과 로딩 대기

        # iframe으로 전환
        iframe = wait_and_find_element(driver, By.ID, "searchIframe")
        if not iframe:
            print("Entry iframe not found")
            return reviews
        
        driver.switch_to.frame(iframe)
        
        # 검색 결과에서 정확한 식당 선택
        restaurant_link = wait_and_find_element(driver, By.CSS_SELECTOR, "a.P7gyV")
        if not restaurant_link:
            print("Restaurant link not found")
            return reviews
        
        restaurant_link.click()

        # 현재 URL 가져오기
        current_url = driver.current_url
        

    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        driver.quit()
    
    return reviews

def save_to_csv(data, filename='naver_map_reviews.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")



restaurant_name = "맛불"  # 검색할 식당 이름
reviews = search_and_extract_reviews(restaurant_name, num_reviews=50)  # 원하는 리뷰 수 지정
save_to_csv(reviews)

Element not found: entryiframe
Entry iframe not found
Data saved to naver_map_reviews.csv


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import csv
import time
import re

def extract_number(text):
    return re.search(r'\d+', text).group() if re.search(r'\d+', text) else ''

def scrape_reviews(url):
    driver = webdriver.Chrome()
    driver.get(url)

    reviews = []
    
    try:
        # 리뷰 더 보기 버튼이 있는 경우 클릭
        while True:
            try:
                more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.fvwqf"))
                )
                more_button.click()
                time.sleep(1)
            except TimeoutException:
                break

        # 리뷰 요소들 찾기
        review_elements = driver.find_elements(By.CSS_SELECTOR, "li.pui__X35jYm")

        # 리뷰 정보 추출
        for element in review_elements:
            try:
                author = element.find_element(By.CSS_SELECTOR, "span.pui__NMi-Dp").text
                content = element.find_element(By.CSS_SELECTOR, "a.pui__xtsQN-").text
                visit_date = element.find_element(By.CSS_SELECTOR, "time").text
                visit_count = element.find_element(By.CSS_SELECTOR, "span.pui__gfuUIT:nth-of-type(2)").text
                
                # 리뷰 수와 사진 수 추출
                stats = element.find_elements(By.CSS_SELECTOR, "span.pui__WN-kAf")
                review_count = extract_number(stats[0].text) if len(stats) > 0 else ''
                photo_count = extract_number(stats[1].text) if len(stats) > 1 else ''
                
                reviews.append({
                    "author": author,
                    "content": content,
                    "visit_date": visit_date,
                    "visit_count": visit_count,
                    "review_count": review_count,
                    "photo_count": photo_count
                })
            except NoSuchElementException:
                continue  # 일부 정보가 없는 경우 해당 리뷰 건너뛰기

    finally:
        driver.quit()

    return reviews

def save_to_csv(reviews, filename="reviews.csv"):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["author", "content", "visit_date", "visit_count", "review_count", "photo_count"])
        writer.writeheader()
        for review in reviews:
            writer.writerow(review)


url = "https://pcmap.place.naver.com/restaurant/1647171960/review/visitor"
reviews = scrape_reviews(url)
save_to_csv(reviews)
print(f"{len(reviews)} 개의 리뷰가 추출되어 reviews.csv 파일에 저장되었습니다.")

47 개의 리뷰가 추출되어 reviews.csv 파일에 저장되었습니다.
